<a href="https://colab.research.google.com/github/kjedrzejczak/Machine_Learning_Bootcamp_practice_II/blob/main/14_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:2f}')

ładowanie danych

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv  # ✨✨✨✨✨✨✨✨✨✨✨✨✨
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2024-04-16 15:41:08--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.207, 142.251.111.207, 142.251.16.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv.1’

products.csv.1      100%[===================>]   2.07M  3.35MB/s    in 0.6s    

2024-04-16 15:41:10 (3.35 MB/s) - ‘products.csv.1’ saved [2166953/2166953]

--2024-04-16 15:41:10--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.207, 142.251.111.207, 142.251.16.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-st

In [3]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])  # tylko dwie kolumny do obliczeń
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [4]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id']) # tylko diw kolumny do oboiczeń
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [5]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head(10)

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado
1382192,1,49683,Cucumber Kirby
325660,1,13176,Bag of Organic Bananas
1375139,1,49302,Bulgarian Yogurt
1179949,36,43086,Super Greens Salad
503846,36,19660,Spring Water


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,1384617.000000,1706297.621090,989732.648868,1.000000,843370.000000,1701880.000000,2568023.000000,3421070.000000
product_id,1384617.000000,25556.235729,14121.272449,1.000000,13380.000000,25298.000000,37940.000000,49688.000000


In [7]:
data['product_name'].value_counts()

product_name
Banana                                                               18726
Bag of Organic Bananas                                               15480
Organic Strawberries                                                 10894
Organic Baby Spinach                                                  9784
Large Lemon                                                           8135
                                                                     ...  
Original Raincoast Oat Crisp                                             1
Petite Creme Lowfat Strawberry Yogurt                                    1
Organic Petite Creme Mon Cherry Amour Yogurt                             1
Greek 100 Calories Whips! Peaches 'n Cream Fat Free Yogurt Mousse        1
Plain Golden Toasted Bread Crumbs                                        1
Name: count, Length: 39123, dtype: int64

In [8]:
data['order_id'].nunique() # liczba uniklanych transakcji

131209

In [9]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

order_id
1          Organic Whole String Cheese,Organic Celery Hea...
36         Super Greens Salad,Spring Water,Organic Garnet...
38         Green Peas,Organic Baby Arugula,Shelled Pistac...
96         Organic Raspberries,Organic Pomegranate Kernel...
98         Uncured Applewood Smoked Bacon,Black Beans,Org...
                                 ...                        
3421049    Gluten Free Rice Bread,Organic Baby Broccoli,O...
3421056    Sparkling Lemon Water,Brioche Buns,Homestyle C...
3421058    Classic Britannia Crisps,Club Soda Lower Sodiu...
3421063    Natural Artesian Water,Organic Half & Half,Twi...
3421070    Broccoli Florettes,Organic Unsweetened Almond ...
Name: product_name, Length: 131209, dtype: object

In [10]:
transactions = transactions.str.split(',')  # otrzymamy listę
transactions

order_id
1          [Organic Whole String Cheese, Organic Celery H...
36         [Super Greens Salad, Spring Water, Organic Gar...
38         [Green Peas, Organic Baby Arugula, Shelled Pis...
96         [Organic Raspberries, Organic Pomegranate Kern...
98         [Uncured Applewood Smoked Bacon, Black Beans, ...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Baby Broccoli...
3421056    [Sparkling Lemon Water, Brioche Buns, Homestyl...
3421058    [Classic Britannia Crisps, Club Soda Lower Sod...
3421063    [Natural Artesian Water, Organic Half & Half, ...
3421070    [Broccoli Florettes, Organic Unsweetened Almon...
Name: product_name, Length: 131209, dtype: object

In [11]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
tarnsactions_encoded = encoder.transform(transactions, sparse=True)  # sparse True zwraca macierz rzadką
tarnsactions_encoded

<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [12]:
transactions_encoded_df = pd.DataFrame(tarnsactions_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Algorytm Apriori

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True)
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

,support,itemsets
8,0.143138,(Banana)
7,0.117980,(Bag of Organic Bananas)
76,0.083028,(Organic Strawberries)
41,0.074568,(Organic Baby Spinach)
31,0.062000,(Large Lemon)
37,0.056467,(Organic Avocado)
61,0.055583,(Organic Hass Avocado)
100,0.049494,(Strawberries)
33,0.045980,(Limes)
69,0.042268,(Organic Raspberries)


In [14]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]] # najlepiej tylko te kolumny
rules = rules.sort_values(by='lift', ascending=False)
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support,confidence,lift
27,(Clementines),( Bag),0.011295,0.523860,36.835557
26,( Bag),(Clementines),0.011295,0.794212,36.835557
23,(Limes),(Large Lemon),0.012156,0.264379,4.264159
22,(Large Lemon),(Limes),0.012156,0.196066,4.264159
18,(Organic Raspberries),(Organic Strawberries),0.012728,0.301118,3.626710


In [15]:
rules = rules.sort_values(by='confidence', ascending=False)
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support,confidence,lift
26,( Bag),(Clementines),0.011295,0.794212,36.835557
27,(Clementines),( Bag),0.011295,0.523860,36.835557
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.018444,0.331825,2.812560
16,(Organic Raspberries),(Bag of Organic Bananas),0.013566,0.320952,2.720400
18,(Organic Raspberries),(Organic Strawberries),0.012728,0.301118,3.626710


In [16]:
rules = rules.sort_values(by='support', ascending=False)
rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support,confidence,lift
1,(Bag of Organic Bananas),(Organic Strawberries),0.023428,0.198579,2.391714
0,(Organic Strawberries),(Bag of Organic Bananas),0.023428,0.282174,2.391714
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.018444,0.331825,2.812560
2,(Bag of Organic Bananas),(Organic Hass Avocado),0.018444,0.156331,2.812560
4,(Organic Baby Spinach),(Bag of Organic Bananas),0.017042,0.228536,1.937082
